In [ ]:
#pip install Recommenders
#pip install --upgrade pip
#pip install ms-recommenders
#pip install ms-recommenders[examples]
%pip install Recommenders 

     |████████████████████████████████| 335 kB 5.2 MB/s 
     |████████████████████████████████| 11.8 MB 46.8 MB/s 
     |████████████████████████████████| 636 kB 65.7 MB/s 
     |████████████████████████████████| 12.4 MB 35.5 MB/s 
     |████████████████████████████████| 310 kB 35.8 MB/s 
     |████████████████████████████████| 197 kB 50.9 MB/s 
     |████████████████████████████████| 1.5 MB 34.8 MB/s 
     |████████████████████████████████| 4.2 MB 45.6 MB/s 
     |████████████████████████████████| 61 kB 8.8 MB/s 
     |████████████████████████████████| 749 kB 51.7 MB/s 
     |████████████████████████████████| 11.1 MB 36.5 MB/s 
     |████████████████████████████████| 384 kB 72.5 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 6.6 MB 52.4 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705395 sha256=054898e09a8aeb600ae7c20c5e342842bcfdd6e90201f21377b4d315357644ed
  Stored in directory: /roo

In [3]:
import sys
#sys.path.append("../../")
import time
import os
import shutil
# import papermill as pm
import pandas as pd
import numpy as np
%tensorflow_version 1.x

import tensorflow as tf
#print(tensorflow.__version__)
from reco_utils.recommender.ncf.ncf_singlenode import NCF
from reco_utils.recommender.ncf.dataset import Dataset as NCFDataset
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_chrono_split
from reco_utils.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from reco_utils.common.constants import SEED as DEFAULT_SEED

# top k items to recommend
TOP_K = 10

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'

# Model parameters
EPOCHS = 100
BATCH_SIZE = 256

SEED = DEFAULT_SEED  # Set None for non-deterministic results

# loading movie leans data
df = movielens.load_pandas_df(size=MOVIELENS_DATA_SIZE,header=["userID", "itemID", "rating", "timestamp"])


NameError: ignored

In [ ]:
df.head()

,userID,itemID,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [ ]:
# 75%비중으로 나누기 
train, test = python_chrono_split(df, 0.75)

# user id, item id ,rating 가 있는 데이터만 가져옴 
data = NCFDataset(train=train, test=test, seed=SEED)

tf.compat.v1.disable_eager_execution()

In [ ]:
model = NCF(n_users=data.n_users,n_items=data.n_items,model_type="NeuMF",n_factors=4,layer_sizes=[16,8,4],\
            n_epochs=EPOCHS,batch_size=BATCH_SIZE,learning_rate=1e-3,verbose=10,seed=SEED)
# n_factors : latent space의 차원 수 (사용자 설정)
# layer_sizes: MLP 레이어 숫자 

# 모델학습 
model.fit(data)

predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]

# 예측값 저장하기 
predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


,userID,itemID,prediction
0,1.0,149.0,0.014823
1,1.0,88.0,0.337588
2,1.0,101.0,0.500791
3,1.0,110.0,0.072838
4,1.0,103.0,0.014223


In [ ]:
users, items, preds = [], [], []
item = list(train.itemID.unique())
for user in train.userID.unique():
    user = [user] * len(item) 
    users.extend(user)
    items.extend(item)
    preds.extend(list(model.predict(user, item, is_list=True)))

all_predictions = pd.DataFrame(data={"userID": users, "itemID":items, "prediction":preds})

merged = pd.merge(train, all_predictions, on=["userID", "itemID"], how="outer")
all_predictions = merged[merged.rating.isnull()].drop('rating', axis=1)

eval_map = map_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_ndcg = ndcg_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_precision = precision_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)
eval_recall = recall_at_k(test, all_predictions, col_prediction='prediction', k=TOP_K)


In [ ]:
eval_precision

0.17826086956521742

In [ ]:
all_predictions['prediction']=round(all_predictions['prediction'],2)

In [ ]:
all_predictions

,userID,itemID,timestamp,prediction
74992,1,286,NaN,0.23
74993,1,258,NaN,0.56
74994,1,305,NaN,0.01
74995,1,307,NaN,0.11
74996,1,288,NaN,0.68
...,...,...,...,...
1505966,943,1592,NaN,0.00
1505967,943,1676,NaN,0.00
1505968,943,907,NaN,0.00
1505969,943,1681,NaN,0.00
